In [2]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def press_cookie(driver):
# O código dentro deste bloco try tentará executar as seguintes ações, que estão relacionadas à busca e aceitação do botão de cookies. 
# 'webdriverwait' = para esperar (até um máximo de x segundos) até que um elemento da página da web seja clicável
    try:
        botao_cookie = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="lgpd-consent-widget"]/section/div/div[2]/button[1]'))  # Change the selector to match your website
        )
        botao_cookie.click()

    except:
        print('Cookie not found')

In [3]:
# usando o CATHO.csv para pegar as urls de todas as vagas:

catho = pd.read_csv('CATHO.csv')

# Só puxando a coluna 'url'

catho.at[0, 'url']

'https://www.catho.com.br/vagas/analista-de-dados/24394457/?origem_apply=busca-de-vagas&entrada_apply=direto'

In [5]:
driver = webdriver.Chrome()

i = 0

while i < len(catho):
    
# Isso inicia um loop while que continuará até que o valor de i seja menor que o comprimento do DataFrame catho

        url = catho.at[i, 'url']
        driver.get(url) # isso instrui o driver do Selenium a carregar a página da web com a URL especificada na variável url
        if i == 0:
                press_cookie(driver)

        # Isso aguarda até que um elemento com a classe CSS 'job-description' seja presente na página (ou por até 10 segundos) 
        # e, em seguida, extrai o texto desse elemento. O texto é armazenado na variável job_description

        job_description = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, 'job-description'))
                ).text
        paragrafos = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.TAG_NAME, 'p'))
                )
        # isso extrai o texto de cada elemento da lista paragrafos e armazena os textos em uma nova lista chamada texto_paragrafos

        texto_paragrafos = [x.text for x in paragrafos]
        texto_paragrafos = ' '.join(texto_paragrafos)

        df = pd.DataFrame(
                {
                        'url': [url],
                        'job_description': [job_description],
                        'paragrafos': [texto_paragrafos]
                }
        )

        

        df.to_csv(f'info_jobs_df/{i}.csv')
        i+=1 # soma +1 após a execução

driver.quit() # fecha o chrome ao final do loop


In [132]:
import pandas as pd
import os

# Caminho para a pasta que contém os arquivos CSV
pasta = 'info_jobs_df'

# Lista todos os arquivos CSV na pasta
arquivos_csv = [os.path.join(pasta, arquivo) for arquivo in os.listdir(pasta) if arquivo.endswith('.csv')]

# Inicialize um DataFrame vazio para armazenar os dados combinados
dados_combinados = pd.DataFrame()

# Loop pelos arquivos CSV e concatene-os
for arquivo in arquivos_csv:
    df = pd.read_csv(arquivo)
    dados_combinados = pd.concat([dados_combinados, df], ignore_index=True)

# Salve o DataFrame combinado em um novo arquivo CSV
dados_combinados.to_csv('dados_combinados.csv', index=False)

pd.read_csv('dados_combinados.csv')

,Unnamed: 0,url,job_description,paragrafos
0,0,https://www.catho.com.br/vagas/analista-de-dad...,Estamos procurando um Analista de Dados para c...,"Tíquete Refeição, Tíquete Alimentação, Vale Tr..."
1,0,https://www.catho.com.br/vagas/analista-de-dad...,""" Elaboração, automação de processos manuais e...","Assistência Médica / Medicina em grupo, Assist..."
2,0,https://www.catho.com.br/vagas/analista-de-dad...,"Responsável por coletar, analisar e transforma...",CLT (Efetivo) ?Assistência Odontológica ?Café ...
3,0,https://www.catho.com.br/vagas/analista-de-dad...,"Coletar, consolidar e analisar dados financeir...",Restaurante na empresa De segunda-feira a Sába...
4,0,https://www.catho.com.br/vagas/analista-de-dad...,Coleta de dados: Identificar e coletar dados r...,Total de anúncios: 93 GRUPO GPS SANTA CRUZ SAN...
...,...,...,...,...
249,0,https://www.catho.com.br/vagas/analista-de-dad...,Inovação é o que nos move. A Cogna é uma das m...,CLT (Efetivo) Modelo de trabalho híbrido (3x n...
250,0,https://www.catho.com.br/vagas/analista-de-dad...,O time de Engenharia da Hype vem buscando cada...,CLT (Efetivo) Exclusividades dos parceiros Hyp...
251,0,https://www.catho.com.br/vagas/analista-de-dad...,Você é conhecedor das estratégias de negócio? ...,CLT (Efetivo) Horário de trabalho: 08h às 12h ...
252,0,https://www.catho.com.br/vagas/analista-de-dad...,""" Extração, Compilação e análise de dados \n"" ...","Assistência Médica / Medicina em grupo, Assist..."


In [133]:
# Juntando os 2 csv - catho + dados_combinados:

# Leia os dois arquivos CSV
df_catho = pd.read_csv('CATHO.csv')
df_dados_combinados = pd.read_csv('dados_combinados.csv')

# Realize a junção (merge) com base na coluna "url"
resultado = df_catho.merge(df_dados_combinados, on='url', how='inner')

# Remova a coluna "Unnamed: 0_y" (ou qualquer outra coluna indesejada)
resultado = resultado.drop(columns=['Unnamed: 0_y', 'Unnamed: 0_x'])

# Salve o resultado em um novo arquivo CSV
resultado.to_csv('dados_compilados.csv', index=False)

pd.read_csv('dados_compilados.csv')

,titulos,empresa,todas_salario,todos_local,url,job_description,paragrafos
0,Analista de Dados,APE SMART TEC,A Combinar,São Paulo - SP (1),https://www.catho.com.br/vagas/analista-de-dad...,Estamos procurando um Analista de Dados para c...,"Tíquete Refeição, Tíquete Alimentação, Vale Tr..."
1,Analista de Dados Pleno,J&T Express Brazil Ltda,"De R$ 5.001,00 a R$ 6.000,00",São Paulo - SP (1),https://www.catho.com.br/vagas/analista-de-dad...,""" Elaboração, automação de processos manuais e...","Assistência Médica / Medicina em grupo, Assist..."
2,Analista de Dados,Engenharia,"De R$ 3.001,00 a R$ 4.000,00",Belo Horizonte - MG (1),https://www.catho.com.br/vagas/analista-de-dad...,Extração de dados. Construção e análise de ind...,"Assistência Médica / Medicina em grupo, Assist..."
3,Analista de Dados,INVESTIDOR SARDINHA,A Combinar,Goiania - GO (1),https://www.catho.com.br/vagas/analista-de-dad...,"Estamos em busca de uma pessoa extraordinária,...",CLT (Efetivo) Contratação CLT;Vaga presencial ...
4,Analista de Dados,BEST TALENTS,A Combinar,São Paulo - SP (1),https://www.catho.com.br/vagas/analista-de-dad...,Empresa de Logística contrata Analista de Dado...,Prestador de serviços (PJ) Horário de Trabalho...
...,...,...,...,...,...,...,...
249,PESSOA ANALISTA DE DADOS SR - Riscos/Inteligência,EMPREGARA,A Combinar,São Paulo - SP (1),https://www.catho.com.br/vagas/pessoa-analista...,Somos uma empresa de tecnologia apaixonada por...,Cadastre-se gratuitamente para ver mais inform...
250,Estágio em Análise de Dados (Home Office),REHAGRO,A Combinar,Belo Horizonte - MG (1),https://www.catho.com.br/vagas/estagio-em-anal...,Oi! Tudo certinho? Que bom te ver por aqui! Vo...,Cadastre-se gratuitamente para ver mais inform...
251,Analista de Banco de Dados,RENOVA RH,"De R$ 3.001,00 a R$ 4.000,00",São Paulo - SP (1),https://www.catho.com.br/vagas/analista-de-ban...,"Limpar e processar dados, incluindo a identifi...","Assistência Médica / Medicina em grupo, Assist..."
252,Analista Banco de Dados,Empresa ConfidencialPor que?,A Combinar,Americana - SP (1),https://www.catho.com.br/vagas/analista-banco-...,""" Análise de dados e requisitos para aplicaçõe...","De se CLT (Efetivo), Prestador de serviços (PJ..."


In [134]:
# Tratamento da planilha no PYTHON:
import numpy as np
import re

texto = 'De R$ 5.001,00 a R$ 6.000,00'

def treat_valores(texto):
    if texto != 'A Combinar':

        texto = texto.replace('De', '')
        texto = texto.replace('R$', '')
        texto = texto.replace('Até', '')
        texto = texto.replace(' ', '')
        texto = texto.replace('.', '')
        texto = texto.replace(',', '.')

        texto_split = texto.split('a')

        texto_split = [float(x) for x in texto_split]

        return np.array(texto_split).mean()
    else:
        return None

resultado['todas_salario'] = resultado['todas_salario'].apply(lambda x: treat_valores(x))

resultado



,titulos,empresa,todas_salario,todos_local,url,job_description,paragrafos
0,Analista de Dados,APE SMART TEC,NaN,São Paulo - SP (1),https://www.catho.com.br/vagas/analista-de-dad...,Estamos procurando um Analista de Dados para c...,"Tíquete Refeição, Tíquete Alimentação, Vale Tr..."
1,Analista de Dados Pleno,J&T Express Brazil Ltda,5500.5,São Paulo - SP (1),https://www.catho.com.br/vagas/analista-de-dad...,""" Elaboração, automação de processos manuais e...","Assistência Médica / Medicina em grupo, Assist..."
2,Analista de Dados,Engenharia,3500.5,Belo Horizonte - MG (1),https://www.catho.com.br/vagas/analista-de-dad...,Extração de dados. Construção e análise de ind...,"Assistência Médica / Medicina em grupo, Assist..."
3,Analista de Dados,INVESTIDOR SARDINHA,NaN,Goiania - GO (1),https://www.catho.com.br/vagas/analista-de-dad...,"Estamos em busca de uma pessoa extraordinária,...",CLT (Efetivo) Contratação CLT;Vaga presencial ...
4,Analista de Dados,BEST TALENTS,NaN,São Paulo - SP (1),https://www.catho.com.br/vagas/analista-de-dad...,Empresa de Logística contrata Analista de Dado...,Prestador de serviços (PJ) Horário de Trabalho...
...,...,...,...,...,...,...,...
249,PESSOA ANALISTA DE DADOS SR - Riscos/Inteligência,EMPREGARA,NaN,São Paulo - SP (1),https://www.catho.com.br/vagas/pessoa-analista...,Somos uma empresa de tecnologia apaixonada por...,Cadastre-se gratuitamente para ver mais inform...
250,Estágio em Análise de Dados (Home Office),REHAGRO,NaN,Belo Horizonte - MG (1),https://www.catho.com.br/vagas/estagio-em-anal...,Oi! Tudo certinho? Que bom te ver por aqui! Vo...,Cadastre-se gratuitamente para ver mais inform...
251,Analista de Banco de Dados,RENOVA RH,3500.5,São Paulo - SP (1),https://www.catho.com.br/vagas/analista-de-ban...,"Limpar e processar dados, incluindo a identifi...","Assistência Médica / Medicina em grupo, Assist..."
252,Analista Banco de Dados,Empresa ConfidencialPor que?,NaN,Americana - SP (1),https://www.catho.com.br/vagas/analista-banco-...,""" Análise de dados e requisitos para aplicaçõe...","De se CLT (Efetivo), Prestador de serviços (PJ..."


In [137]:
#------------------------#
# instalar: pip install pandas openpyxl // para salvar em Excel


local = 'São Paulo - SP (1)'

def treat_valores(local):
    local = local.replace('(1)', '')
    local = re.sub(r'\(\d+\)', '', local)
    local_split = local.split('-')
    local_split = [str(x) for x in local_split]

    return local_split[0]

resultado['Cidade'] = resultado['todos_local'].apply(lambda x: treat_valores(x))

def treat_valores(local):
    local = re.sub(r'\(\d+\)', '', local)
    local_split = local.split('-')
    local_split = [str(x) for x in local_split]

    return local_split[1]

resultado['Estado'] = resultado['todos_local'].apply(lambda x: treat_valores(x))

resultado.to_csv('CATHO_FINAL.csv')